# Detecting Objects using the Wallaroo faster-rcnn Pipeline in a Video Stream

In this tutorial we will explore using the mobilenet pipeline we created in step 3 to run inference on the frames in a Video and then draw the identified object's bounding boxes, classification and classification confidence.

In [1]:
!pip install imutils

import torch
import pickle
import wallaroo
import os
import numpy as np
import json
import requests
import time
import imutils
from CVDemoUtils import CVDemo


/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wl = wallaroo.Client()

In [3]:
ws = wl.list_workspaces()
for w in ws:
    if w.name() == 'computer-vision':
        wl.set_current_workspace(w)

In [4]:
model_name = 'resnet50'
onnx_model_path = 'models/frcnn-resnet.pt.onnx'
resnet_model = wl.upload_model(model_name, onnx_model_path)

In [5]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(2).memory("12Gi").build()

In [6]:
pipeline_name = 'resnet-pp'
pipeline = wl.build_pipeline('resnet-pp') \
            .add_model_step(resnet_model) \
            .deploy(deployment_config = deployment_config)

 ok


In [7]:
time.sleep(5) # needed to allow the pipeline to settle in.
url = pipeline._deployment._url()
print(url)

http://engine-lb.resnet-pp-971:29502/pipelines/resnet-pp


## Test the pipeline in a video stream

### Detect and Classify Objects in Video using Wallaroo Shadow Deployment

Each frame in the input-video will be fed to the pipeline for inferencing

In [8]:
# The size the image will be resized to
width = 640
height = 480

cvDemo = CVDemo()

input_video = "videos/amazon-fresh-go.mp4"
output_video = "videos/amazon-fresh-go-resnet50-inferenced.mp4"

#input_video = "videos/ww2-warbirds-in-formation.mp4"
#output_video = "videos/ww2-warbirds-in-formation-inferenced.mp4"

config = {
    'input-video' : input_video, # source video
    'output-video' : output_video, # show the input video with the inferenced results drawn on each frame
    'fps' : 15, # Frames per second
    'endpoint-url' : url, # the pipelines rest api endpoint url
    'width' : width, # the width of the url
    'height' : height, # the height of the url
    #'max-frames' : 400, # the # of frames to capture in the output video
    'skip-frames' : 150, # the # of frames to capture in the output video
    'confidence-target' : 0.90, # only display bounding boxes with confidence > provided #
    'color': CVDemo.CYAN, # color to draw bounding boxes and the text in the statistics
    'inference' : 'WALLAROO_SDK', # "ONNX" or "WALLAROO_API" or "WALLAROO_SDK"
    'onnx_model_path' : onnx_model_path,
    'model_name' : model_name,
    'pipeline' : pipeline, # provide this when using inference WALLAROO_SDK 
    'pipeline_name' : pipeline_name
}
cvDemo.DEBUG = False
cvDemo.detectAndClassifyObjectsInVideo(config)
print("We are done.")


11-11-2022 11:58:31.533845 Start Time:06:58:31 AM
11-11-2022 11:58:31.533916 Skipping [150] frames
11-11-2022 11:58:31.544183 Video Properties
11-11-2022 11:58:31.544256    video input:videos/amazon-fresh-go.mp4
11-11-2022 11:58:31.544272    video output:videos/amazon-fresh-go-resnet50-inferenced.mp4
11-11-2022 11:58:31.544313    format:0.0
11-11-2022 11:58:31.544330    fourcc:828601953.0
11-11-2022 11:58:31.544344    mode:0.0
11-11-2022 11:58:31.544355    buffer:0.0
11-11-2022 11:58:31.544370    width:1280.0
11-11-2022 11:58:31.544382    height:720.0
11-11-2022 11:58:31.544395    fps:25.0
11-11-2022 11:58:31.544462    frame size:(640, 630)
11-11-2022 11:58:31.545754 recordStartFrame:0
11-11-2022 11:58:31.545788 recordEndFrame:0
11-11-2022 11:58:35.278276 Frame:150 Read: 0.0011 Inf: 3.1231 Onnx: 0.0007 Draw: 0.0059 Total: 3.2335
11-11-2022 11:58:39.012735 Frame:151 Read: 0.0015 Inf: 3.6832 Onnx: 0.0003 Draw: 0.0067 Total: 3.7340
11-11-2022 11:58:42.653583 Frame:152 Read: 0.0012 Inf: 3.

In [9]:
#pipeline.undeploy()
#for d in wl.list_deployments():
#    d.undeploy()
    

Please be patient i

### Conclusion

Notice how simple it is to take the mobilenet object detectors rest api endpoint url and use it in a video stream.  Now its your turn.  Upload a video to this notebook and replace the input-video with the path of the uploaded video.  Update the output video accordingly.

See how well it works for you.  Good luck!


In [16]:
from datetime import timedelta
delta = timedelta(microseconds=51461)
int(51461) / 1e+6

0.051461